# Counterfactuals
## Current model

### Load current model

In [1]:
import joblib


model = joblib.load(filename="../../saved_models/current_model.pkl")

### Load dataset

In [2]:
import numpy as np
import pandas as pd
import sys

import os
import pathlib

sys.path.append("../..")
from training.creating_dataset import load_and_preprocess_data

current_path = os.getcwd()
DATASETS_DIR = pathlib.Path(current_path).parent.parent / "pathfinder_2e_data"
DATASET_FILES = [
    "pathfinder-bestiary.db",
    "pathfinder-bestiary-2.db",
    "pathfinder-bestiary-3.db",
]
DATASET_PATHS = [f"{DATASETS_DIR}/{file}" for file in DATASET_FILES]
characteristics = [
    "cha",
    "con",
    "dex",
    "int",
    "str",
    "wis",
    "ac",
    "hp",
]

df = load_and_preprocess_data(
    DATASET_PATHS,
    characteristics=[
        "cha",
        "con",
        "dex",
        "int",
        "str",
        "wis",
        "ac",
        "hp",
    ],
)

In [3]:
df.head()

,dex,hp,ac,cha,con,str,wis,book,int,level
0,2,215,29,1,5,7,2,Pathfinder Bestiary,1,10
1,0,220,28,1,5,9,2,Pathfinder Bestiary,-4,10
2,3,175,25,-2,6,7,3,Pathfinder Bestiary,-4,8
3,4,315,41,6,5,7,5,Pathfinder Bestiary,6,17
4,3,20,16,1,1,-5,1,Pathfinder Bestiary,-1,1


In [4]:
from serving.backend.constants import ORDERED_CHARACTERISTICS


df = df[ORDERED_CHARACTERISTICS + ["book", "level"]]

In [5]:
df.head()

,str,dex,con,int,wis,cha,ac,hp,book,level
0,7,2,5,1,2,1,29,215,Pathfinder Bestiary,10
1,9,0,5,-4,2,1,28,220,Pathfinder Bestiary,10
2,7,3,6,-4,3,-2,25,175,Pathfinder Bestiary,8
3,7,4,5,6,5,6,41,315,Pathfinder Bestiary,17
4,-5,3,1,-1,1,1,16,20,Pathfinder Bestiary,1


### Counterfactuals

In [6]:
dataset = df.drop(columns=["book"])
continuous_features = dataset.drop(columns=["level"]).columns.tolist()

In [7]:
continuous_features

['str', 'dex', 'con', 'int', 'wis', 'cha', 'ac', 'hp']

In [8]:
dataset.head()

,str,dex,con,int,wis,cha,ac,hp,level
0,7,2,5,1,2,1,29,215,10
1,9,0,5,-4,2,1,28,220,10
2,7,3,6,-4,3,-2,25,175,8
3,7,4,5,6,5,6,41,315,17
4,-5,3,1,-1,1,1,16,20,1


In [9]:
# dice imports
import dice_ml
from dice_ml import Dice

C:\Users\jolka\OneDrive\Dokumenty\Inzynierka\ML-for-TTRPG-game-design\venv\lib\site-packages\dice_ml\utils\exception.py:12: UserWarning: UserConfigValidationException will be deprecated from dice_ml.utils. Please import UserConfigValidationException from raiutils.exceptions.
  warnings.warn("UserConfigValidationException will be deprecated from dice_ml.utils. "


In [10]:
d = dice_ml.Data(
    dataframe=dataset, continuous_features=continuous_features, outcome_name="level"
)

m = dice_ml.Model(model=model, backend="sklearn", model_type="regressor")

In [17]:
exp = Dice(d, m, method="kdtree")

In [13]:
sys.path.append("../../serving/backend")
from serving.backend.calculate_level import calculate_level


query = dataset.drop(columns=["level"])[10:11]
query_dict = query.loc[10].to_dict()
query

,str,dex,con,int,wis,cha,ac,hp
10,3,2,2,-4,1,-1,17,17


In [14]:
dataset.loc[10]

str       3
dex       2
con       2
int      -4
wis       1
cha      -1
ac       17
hp       17
level     1
Name: 10, dtype: int64

In [15]:
level = calculate_level(monster_stats=query_dict, model=model)
level

1

In [18]:
genetic = exp.generate_counterfactuals(
    query, total_CFs=5, desired_range=[-0.67, 0.33], verbose=True
)

100%|██████████| 1/1 [00:00<00:00,  1.36it/s]

Diverse Counterfactuals found! total time taken: 00 min 00 sec


In [19]:
genetic.visualize_as_dataframe(show_only_changes=True)

Query instance (original outcome : 1)


,str,dex,con,int,wis,cha,ac,hp,level
0,3,2,2,-4,1,-1,17,17,1.000145



Diverse Counterfactual set (new outcome: [-0.67, 0.33])


,str,dex,con,int,wis,cha,ac,hp,level
1155,-,-,3.0,-,-,1.0,15.0,18.0,-6.624720117542893e-05
1010,-,-,-,-5.0,2.0,0.0,16.0,15.0,2.3886730559752323e-05
248,-,-,3.0,-,2.0,0.0,14.0,16.0,0.00014299523900263011
284,-,-,3.0,-1.0,-,0.0,15.0,15.0,-0.00015228852862492204
518,2.0,3.0,-,-,3.0,0.0,15.0,16.0,0.0001631952472962439


In [20]:
js = genetic.to_json()

In [21]:
js

'{"test_data": [[[3.0, 2.0, 2.0, -4.0, 1.0, -1.0, 17.0, 17.0, 1.0001450258428668]]], "cfs_list": [[[3.0, 2.0, 3.0, -4.0, 1.0, 1.0, 15.0, 18.0, -6.624720117542893e-05], [3.0, 2.0, 2.0, -5.0, 2.0, 0.0, 16.0, 15.0, 2.3886730559752323e-05], [3.0, 2.0, 3.0, -4.0, 2.0, 0.0, 14.0, 16.0, 0.00014299523900263011], [3.0, 2.0, 3.0, -1.0, 1.0, 0.0, 15.0, 15.0, -0.00015228852862492204], [2.0, 3.0, 2.0, -4.0, 3.0, 0.0, 15.0, 16.0, 0.0001631952472962439]]], "local_importance": null, "summary_importance": null, "data_interface": {"outcome_name": "level", "data_df": "dummy_data"}, "feature_names": ["str", "dex", "con", "int", "wis", "cha", "ac", "hp"], "feature_names_including_target": ["str", "dex", "con", "int", "wis", "cha", "ac", "hp", "level"], "model_type": "regressor", "desired_class": "opposite", "desired_range": [-0.67, 0.33], "metadata": {"version": "2.0"}}'

In [22]:
genetic.cf_examples_list[0].final_cfs_df

,str,dex,con,int,wis,cha,ac,hp
1155,3.0,2.0,3.0,-4.0,1.0,1.0,15.0,18.0
1010,3.0,2.0,2.0,-5.0,2.0,0.0,16.0,15.0
248,3.0,2.0,3.0,-4.0,2.0,0.0,14.0,16.0
284,3.0,2.0,3.0,-1.0,1.0,0.0,15.0,15.0
518,2.0,3.0,2.0,-4.0,3.0,0.0,15.0,16.0


In [23]:
import json


cfs_list = json.loads(js)["cfs_list"][0]

In [24]:
cfs_list

[[3.0, 2.0, 3.0, -4.0, 1.0, 1.0, 15.0, 18.0, -6.624720117542893e-05],
 [3.0, 2.0, 2.0, -5.0, 2.0, 0.0, 16.0, 15.0, 2.3886730559752323e-05],
 [3.0, 2.0, 3.0, -4.0, 2.0, 0.0, 14.0, 16.0, 0.00014299523900263011],
 [3.0, 2.0, 3.0, -1.0, 1.0, 0.0, 15.0, 15.0, -0.00015228852862492204],
 [2.0, 3.0, 2.0, -4.0, 3.0, 0.0, 15.0, 16.0, 0.0001631952472962439]]

In [25]:
sorted(cfs_list, key=lambda x: abs(0 - x[-1]), reverse=True)

[[2.0, 3.0, 2.0, -4.0, 3.0, 0.0, 15.0, 16.0, 0.0001631952472962439],
 [3.0, 2.0, 3.0, -1.0, 1.0, 0.0, 15.0, 15.0, -0.00015228852862492204],
 [3.0, 2.0, 3.0, -4.0, 2.0, 0.0, 14.0, 16.0, 0.00014299523900263011],
 [3.0, 2.0, 3.0, -4.0, 1.0, 1.0, 15.0, 18.0, -6.624720117542893e-05],
 [3.0, 2.0, 2.0, -5.0, 2.0, 0.0, 16.0, 15.0, 2.3886730559752323e-05]]

In [26]:
prev_diff = 0
for cf in cfs_list:
    cf_monster_stats = pd.DataFrame(
        data={char: [cf[i]] for i, char in enumerate(ORDERED_CHARACTERISTICS)}
    )
    print(cf[:-1])
    print(model.predict(cf_monster_stats))
    assert prev_diff <= abs(0 - model.predict(cf_monster_stats))

[3.0, 2.0, 3.0, -4.0, 1.0, 1.0, 15.0, 18.0]
[-6.62472027e-05]
[3.0, 2.0, 2.0, -5.0, 2.0, 0.0, 16.0, 15.0]
[2.38867297e-05]
[3.0, 2.0, 3.0, -4.0, 2.0, 0.0, 14.0, 16.0]
[0.000143]
[3.0, 2.0, 3.0, -1.0, 1.0, 0.0, 15.0, 15.0]
[-0.00015229]
[2.0, 3.0, 2.0, -4.0, 3.0, 0.0, 15.0, 16.0]
[0.0001632]
